# Supply Chain Data Analytics

Analyzing and Forcasting Supermarket Sales

Stan Brouwer (Vrije Universiteit)  
Liz Chan (Master TSCM)  
Maaike Lamberst (Supply Chain Data analysis)  
Niek Schroor (Group 10)  
December 7, 2024

Introduction

We analyze, forecast and interpret the [Superstore sales](https://public.tableau.com/app/sample-data/sample_-_superstore.xls) provided by [Tableau](https://public.tableau.com/app/learn/sample-data) using different statistical and machine learning methods.

We describe our work in the PDF version. However, we would like to recommend reading our quarto manuscript *here* as it contains the **relevant** R code in the Article Notebook.

## 1 Data Pre-processing

The superstore data set we selected is of high quality. Thus we do the required data pre-processing, but included the hypothetical steps we would take were our data of lower quality to communicate our understanding of the data pre-processing process.

We took the following pre-processing steps:

In [ ]:
# Clear workspace
rm(list = ls())
# Function to load (and install if necessary) dependencies
install_and_load <- function(packages) {
  install.packages(setdiff(packages, rownames(installed.packages())), dependencies = TRUE)
  invisible(lapply(packages, require, character.only = TRUE))
}
install_and_load(c("tidyverse", "readxl", "ggplot2", "lubridate", "stats", "Amelia","forecast", "tseries", "plotly", "stringr", "knitr"))

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: readxl

Loading required package: Amelia

Loading required package: Rcpp
## 
## Amelia II: Multiple Imputation
## (Version 1.8.3, built: 2024-11-07)
## Copyright (C) 2005-2024 James Honaker, Gary King and Matthew Blackwell
## Refer to http://gking.harvard.edu/amelia/ for more information
## 
Loading required package: forecast

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Loading required package: tseries

Loading required package: plotly

Attaching package: 'plotly'

The following object is masked from 'package:ggplot2':

    last_plot

The following object is masked from 'package:stats':

    filter

The following object is masked from 'package:graphics':

    layout

Loading required package: knitr

-   Improved column names by removing whitespaces
-   Removed the <code>Row_ID</code> column as it can be inferred by it’s index
-   Removed all columns with a single unique value, as storing these would be [redundant](https://few.vu.nl/~molenaar/courses/StatR/chapters/B-06-raw_data.html)
-   Ensured machine-readable date formats in yyyy-mm-dd as these usually differ per locale.
-   Ensured proper decimal separators
-   Calculated the number of missing values (both NA and empty string ““) per column.

In [ ]:
# Load the data
suppressWarnings({data <- read_excel("data/sample_-_superstore.xls")}) # The Postal code column is stored as 'text' but coerced to numeric, causing warnings which we suppress

# Improve column names
colnames(data) <- str_replace_all(colnames(data), " ", "_")
colnames(data) <- str_replace_all(colnames(data), "-", "_")

# Remove the 'Row_ID' column as it can be inferred by it's index
data <- subset(data, select = -`Row_ID`)

# Remove all columns that have only one unique value, as storing these would be redundant
data <- data[, sapply(data, function(col) length(unique(col)) > 1)]

# Ensure a machine-readable date format as these are usually horrible in excel files
data$Order_Date <- as.Date(data$Order_Date, format = "%Y-%m-%d")
data$Ship_Date <- as.Date(data$Ship_Date, format = "%Y-%m-%d")

# The readxl package by default uses the correct decimal separator (as opposed to base R)

# Calculate the number of missing values per column.
# Origional dates and R date objects are in unix time, which return NA when compared to text (empty string). These dates are stored as 'double' datatype, Thus we check character columns for empty strings, and all columns for NA values. 
missing_values <- sapply(data, function(col) {
  if (inherits(col, "Date")) {
    sum(is.na(col))
  } else if (is.character(col)) {
    sum(is.na(col) | col == "")
  } else {
    sum(is.na(col))
  }
})


if (sum(missing_values) == 0) {
  print("None of the columns contains missing values") # We print to enforce the "Source: Article notebook
  } else {
  print("Some columns contain missing values")
}

[1] "None of the columns contains missing values"

Warning in rm(missing_value, missing_values_table): object 'missing_value' not
found

After these steps (and transposing the table for better document formatting), the data looks as follows:

In [ ]:
kable(t(head(data, 3)), caption = "First 5 Rows of the Data (Transposed)", format = "markdown")

There is some more processing to do, for instance the removal of outliers. However, by doing so we impose our own assumptions on the data. Let’s start by evaluating the descriptive statistics of our data and check if further processing is required.

In [ ]:
descriptive_statistics <- function(column) {
  if (is.numeric(column)) {
    stats <- list(
      Min = min(column, na.rm = TRUE), # Note that handling NA values increases robustness (and I copied the funciton from some of my earlier work)
      Max = max(column, na.rm = TRUE),
      Mean = mean(column, na.rm = TRUE),
      Median = median(column, na.rm = TRUE),
      StdDev = sd(column, na.rm = TRUE)
    )
  } else if (inherits(column, "Date")) {
    stats <- list(
      Earliest = format(min(column, na.rm = TRUE), "%Y-%m-%d"),
      Latest = format(max(column, na.rm = TRUE), "%Y-%m-%d")
    )
  } else if (is.character(column)) {
    stats <- list(
      Unique = length(unique(column)),
      Mode = names(sort(table(column), decreasing = TRUE)[1])
    )
  } else {
    stats <- NULL
  }
  return(stats)
}

# Call function on dataframe
descriptive_stats <- lapply(data, descriptive_statistics)

# Separate to tables dependent on data type
numeric_stats <- as.data.frame(do.call(rbind, lapply(names(data), function(col_name) {
  if (is.numeric(data[[col_name]])) {
    c(Column = col_name, descriptive_stats[[col_name]])
  }
})), stringsAsFactors = FALSE)
date_stats <- as.data.frame(do.call(rbind, lapply(names(data), function(col_name) {
  if (inherits(data[[col_name]], "Date")) {
    c(Column = col_name, descriptive_stats[[col_name]])
  }
})), stringsAsFactors = FALSE)
character_stats <- as.data.frame(do.call(rbind, lapply(names(data), function(col_name) {
  if (is.character(data[[col_name]])) {
    c(Column = col_name, descriptive_stats[[col_name]])
  }
})), stringsAsFactors = FALSE)

# Output tables
kable(
  numeric_stats,
  caption = "Descriptive Statistics for Numeric Columns",
  format = "pipe")

We inspected the orders with the lowest and highers price (<code>Sales</code> in USD). The most expensive orders were professional printers, camera’s and teleconferencing units with high unit prices, and these orders often were of high <code>Quantity</code>. The orders with the lowest price where often binders, had a high <code>Discount</code> rate, and often a <code>Quantity</code> of just one.

We were fascinated by the orders with a negative <code>profit</code>. These all had high <code>Discount</code> rates, and often concerned the same items, such as the <code>Cubify CubeX 3D Printer Triple Head Print</code>. The orders with a negative <code>Profit</code> where often part of a larger order (for instance <code>CA-2016-108196</code>), and placed by customers that placed multiple orders. We suspect these negative <code>Profit</code>’s to be caused by faulty items that receive discounts, general discount codes, or volume discounts. However, due to especially the high discounts on orders with negative profits, we assume these to be valid orders. This decision has also been influenced by the high quality of the data. As we found no missing values whats however, we suspect the chance of some weird but valid orders to be higher than encountering mistakes here. *\[this paragraph could use some rewriting\]*

In figure x we plotted the sales of the most popular products. Unfortunately, the sales of individual products were too low to determine any meaningfull trends.

In [ ]:
# Optionally: print top 10 sale quantity barplot
# # Sum of Quantity for top products
# top_products <- data %>%
#   group_by(Product_Name) %>%
#   summarize(total_quantity = sum(Quantity, na.rm = TRUE)) %>%
#   arrange(desc(total_quantity)) %>%
#   slice_head(n = 10) %>% 
#   mutate(ProdName8 = substr(Product_Name, 1, 8)) # Truncate product names to the first 8 characters. Long names mess up formatting
# 
# # Plot
# ggplot(top_products, aes(x = reorder(ProdName8, -total_quantity), y = total_quantity)) +
#   geom_bar(stat = "identity", fill = "steelblue") +
#   labs(title = "Top 20 Most Sold Products",
#        x = "Product ID",
#        y = "Total Quantity") +
#   theme_minimal() +
#   coord_flip()

# Aggregate quantity by Product Name and Order Date to create a time series
time_series_data <- data %>%
  group_by(Product_Name, Order_Date) %>%
  summarize(total_quantity = sum(Quantity, na.rm = TRUE)) %>%
  ungroup()

`summarise()` has grouped output by 'Product_Name'. You can override using the
`.groups` argument.

ℹ Please use `linewidth` instead.

Our proposed workaround is to aggregate products by their <code>Sub_Category</code>, and treating them as a single product for the rest of the assignment, which we plotted in figure X.

In [ ]:
# Bar plots

# # Count frequency of top 20 products
# top_products <- data %>%
#   count(Product_Name, sort = TRUE) %>%
#   top_n(20, n) %>%
#   mutate(ProdName8 = substr(Product_Name, 1, 8))
# 
# # Plot!
# ggplot(top_products, aes(x = reorder(`ProdName8`, -n), y = n)) +
#   geom_bar(stat = "identity", fill = "steelblue") +
#   labs(title = "Top 20 Most Sold Products",
#        x = "Product Name",
#        y = "Quantity sold") +
#   theme_minimal() +
#   coord_flip()
# 
# Count frequency of top 20 products
top_categories <- data %>%
  count(Sub_Category, sort = TRUE)
# 
# # Plot!
# ggplot(top_categories, aes(x = reorder(Sub_Category, -n), y = n)) +
#   geom_bar(stat = "identity", fill = "steelblue") +
#   labs(title = "Sub_Categories sorted",
#        x = "Product Name",
#        y = "Quantity sold") +
#   theme_minimal() +
#   coord_flip()

# Find top 10 most sold product names
top_10_categories <- top_categories$Sub_Category[0:10]

# Filter the data for  top 10 products
top_10_data <- data %>% filter(Sub_Category %in% top_10_categories)

# calculate sales per month
top_10_data <- top_10_data %>%
  mutate(Month = floor_date(Order_Date, unit = "month"))

# Aggregate data by month for each sub-category
top_10_data_aggregated <- top_10_data %>%
  group_by(Month, Sub_Category) %>%
  summarise(Sales_Count = n(), .groups = 'drop')

# Some special interactive plot formatting (see previous plot)
p_ly <- ggplotly(ggplot(top_10_data_aggregated, aes(x = Month, y = Sales_Count, color = Sub_Category, group = Sub_Category)) +
    geom_line(size = 1) +
    geom_point(size = 2) +
    labs(title = "Monthly Sales for the Top 3 Most Sold Products",
         x = "Month",
         y = "Sales Count",
         color = "Product Name") +
    theme_minimal())

# Modify the visibility of traces
for (i in seq_along(p_ly$x$data)) {
  if (i == 1) {
    p_ly$x$data[[i]]$visible <- TRUE  # Make the first trace visible
  } else {
    p_ly$x$data[[i]]$visible <- "legendonly"  # Hide the rest
  }
}

# Plot
p_ly

These aggregated sales start to show trends and seasonality, and are much more useful to base predictions on! We will use these aggregated sub-categories for the rest of the assignment.

To properly finish our data pre-processing we ran some statistics on the aggregated sub-category sales. Table x contains soem descriptive statistics.

In [ ]:
library(dplyr)
library(kableExtra)


Attaching package: 'kableExtra'

The following object is masked from 'package:dplyr':

    group_rows

The statistics for the sales aggregated by product category look valid. We can further inspect them by visualizing them as histogram and visually check for anomalies. Figure y contains histograms of the quantities per sub-category.

In [ ]:
sub_categories <- unique(data$Sub_Category)

p <- plot_ly()
for (i in seq_along(sub_categories)) {
  sub <- sub_categories[i]
  subset_data <- data %>% filter(Sub_Category == sub)
  p <- add_trace(
    p,
    x = subset_data$Quantity,
    type = "histogram",
    name = sub,
    visible = ifelse(i == 1, TRUE, FALSE)
  )
}

# We add a drop down menu for Sub_Category as toggling visibility in default ggplot2 adds the histograms up. Instead we want to be able to show each histogram seperately. 
dropdown_buttons <- lapply(seq_along(sub_categories), function(i) {
  list(
    method = "update",
    args = list(
      list(visible = lapply(seq_along(sub_categories), function(j) j == i)),
      list(xaxis = list(title = "Quantity", autorange = TRUE), 
           yaxis = list(title = "Frequency", autorange = TRUE))
    ),
    label = sub_categories[i]
  )
})

# Style drop down layout
p <- p %>%
  layout(
    title = "Distribution of Quantity Sold per Order by Sub-Category",
    xaxis = list(title = "Quantity"),
    yaxis = list(title = "Frequency"),
    showlegend = FALSE,  # Drop down instead of legend
    updatemenus = list(
      list(
        type = "dropdown",
        buttons = dropdown_buttons,
        direction = "down",
        x = 0.99,
        y = 0.99,
        showactive = TRUE,
        xanchor = "left",
        yanchor = "top"
      )
    )
  )
p

The histograms show that the quantities are not normally distributed, but have a right-skewed distribution. This is expected as most orders contain a small number of items, but some orders contain a large number of items. We will not remove these outliers as they are valid orders.

As the data we are going to use seems valid, we move on to exploring the trends and visualizing our data.

## 2 Data Visualization

some text for the visualization